# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = '/home/robin/GIT/ADA2017-Tutorials/02 - Intro to Pandas/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
print(DATA_FOLDER)

In [ ]:
import pandas as pd
import numpy as np
import re

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
# Write your answer here

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

First we are going to concatenate all excel files into one `DataFrame`. We'll make sure we have a unique index by resetting it to a temporary 0-x numerical index. We're going to be using the variable `xl_lengths` to keep track of the size of the different tables that are concatenated.

In [ ]:
mb_data = pd.DataFrame(columns=['raw','counts'])
nb_excels = 9
counts = 0
xl_lengths = np.zeros(nb_excels, dtype=int)

# Concat all excel files
for i in range(nb_excels):
    mb_new = pd.read_excel(DATA_FOLDER+"/microbiome/MID"+str(i+1)+".xls", header=None, names=['raw','counts'])
    #mb_new.index += np.size(mb_data.index)
    xl_lengths[i] = np.size(mb_new.index)
    counts += mb_new['raw'].size
    mb_data = pd.concat([mb_data, mb_new], axis=0, join='outer')

# Reset indices (df.reset_)
mb_data = mb_data.reset_index() # why does it create a new 'index' column?
mb_data = mb_data.drop('index', axis=1) # dropping this new column

print('Index is unique: ', mb_data.index.is_unique)
print('Nb. of data points:', counts)
print('Data frame shape', mb_data.shape)
mb_data.head()

We read in the metadata :

In [ ]:
mb_metadata = pd.read_excel(DATA_FOLDER+"/microbiome/metadata.xls")
print('Metadata DF index is unique:',mb_metadata.index.is_unique)
mb_metadata.head(3)

We now initialise the new columns that will be filled in with the metadata information

In [ ]:
mb_data = pd.concat([mb_data, pd.DataFrame(columns=mb_metadata.columns)], axis=1)
print('Data frame shape:',mb_data.shape)
mb_data.head()

We now add the metadata to the corresponding slices of the overall `DataFrame` using our previous `xl_length` variable to keep track of when the imported tables end and another begins.

In [ ]:
idx = 0
for j in range(np.size(xl_lengths)):
    if j >= 1:
        idx = sum(xl_lengths[0:j])
    for i in mb_metadata.columns:
        if j == 0:
            mb_data[i][0:xl_lengths[j]+1] = mb_metadata[i][j]
        else:
            mb_data[i][idx:idx+xl_lengths[j]+1] = mb_metadata[i][j]
#print(mb_data[mb_metadata.columns])#[0:xl_lengths[0]+10])

print(xl_lengths)
#print(sum(xl_lengths)-xl_lengths[8])
print('Final DataFrame index is unique', mb_data.index.is_unique)
mb_data.head()

We now create a meaningful index using all the metadata that we imported and joined together:

In [ ]:
new_index = ['raw', 'BARCODE', 'GROUP', 'SAMPLE']
mb_data = mb_data.set_index(new_index)
#type(mb_metadata.columns)
mb_data.head()

In [ ]:
mb_data.loc['Archaea "Crenarchaeota" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera']

Depending on how we wish to view the combined data frame, for example all tissue samples of a specific bacteria, we can swap indices to make this easier:

In [ ]:
mb_data.swaplevel('BARCODE','SAMPLE').loc['Archaea "Crenarchaeota" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera']

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_excel(DATA_FOLDER+'/titanic.xls', sheetname='titanic', header=0)

## For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

### Task 3.1 

In [ ]:
# Question: Describe the type and the value range of each attribute. 

# We assume that we can describe the value range of each attribute
# only if it's a numerical type

types = df.dtypes
value_range = df._get_numeric_data().apply(lambda x: (x.min(), x.max()))

types.index.name = 'Attribute'
types.columns = 'Type'

value_range.index.name = 'Attribute'
value_range.columns = 'Range'

res = pd.concat(dict(Range=value_range, Type=types), axis=1).fillna('-')
res



In [ ]:
# Question: Indicate and transform the attributes that can be Categorical.

# By visual inspection we can argue that the following columns could be
# categorical : 'pclass', 'survived', 'sex', 'sibsp', 'parch', 'embarked'
#
# We assumed that for an attribute to be categorical it should have 
# <= 10 distinct elements. (NaN is not considered as a distinct value)


categories = [v for v in df.columns if (pd.Series(['unique']).isin(df[v].describe().index.tolist())).bool()
                                            and df[v].describe()['unique'] < 10 ]


df['pclass'] = df.pclass.astype('category')
df['survived'] = df.survived.astype('category')
df['sex'] = df.sex.astype('category')
df['sibsp'] = df.sibsp.astype('category')
df['parch'] = df.parch.astype('category')
df['embarked'] = df.embarked.astype('category')

### Task 3.2

In [ ]:
# Question: Plot a histogram for the travel class attribute
%matplotlib inline

xs = [1, 2, 3]
l = df['pclass'].tolist()
ys = [l.count(1), l.count(2), l.count(3)]

temp = []
[temp.append(v) for v in df['pclass'].index if not df['pclass'][v] in temp]


'''hist_plot = df.pclass.hist(bins=3, grid=False, xlabelsize=11, ylabelsize=11, figsize=(10, 6))
hist_plot.set_title('Travel Class')
hist_plot.set_xlabel('Class')
hist_plot.set_ylabel('Passengers')'''



In [ ]:
# Question: Plot a histogram for the embarkation port attribute
%matplotlib inline

nb_ports = df['embarked'].describe()['unique']
hist_plot = df.embarked.hist(bins=3, grid=False, xlabelsize=11, ylabelsize=11, figsize=(6, 6))
hist_plot.set_title('Embarkation port')
hist_plot.set_xlabel('Port')
hist_plot.set_ylabel('Passengers')

In [ ]:
# Question: Plot a histogram for the sex attribute
%matplotlib inline

hist_plot = df.pclass.hist(bins=2, grid=False, xlabelsize=11, ylabelsize=11, figsize=(10, 6))
hist_plot.set_title('Sex')
hist_plot.set_xlabel('Sex')
hist_plot.set_ylabel('Passengers')

In [ ]:
# Question: Plot a histogram for the age attribute


hist_plot = df.age.hist(bins=3, grid=False, xlabelsize=11, ylabelsize=11, figsize=(10, 6))
hist_plot.set_title('Age')
hist_plot.set_xlabel('Ages')
hist_plot.set_ylabel('Passengers')


### Task 3.3

In [ ]:
# Question: Calculate the proportion of passengers by cabin floor. 
# Present your results in a pie chart.

# We assumed that the information on the cabin floor is available 
# in the column 'cabin' where the letter indicates the floor.
# the number that follows indicates a specific cabin on that floor.

# Unfortunately, this column is also filled with NaN's values.
# For this statistic we won't try guessing the cabin floor for 
# passenger where the data is not provided. Our analysis relies
# only on values that are already in the dataset.

tot = df.cabin.describe()['count']

temp = df.cabin.dropna(axis=0).apply(lambda x: str(x)[:1])

D = {}
for v in temp:
    if not v in D:
        D[v] = 1
    else:
        D[v] += 1


labels = list(D.keys())
values = list(D.values())


plt.pie(values, labels=labels)
plt.show()


### Task 3.4

In [ ]:
# Question: For each travel class, calculate the proportion of the
# passengers that survived. Present your results in pie charts.

tot = df.survived.describe()['count']

class_tot = (df.groupby('pclass').survived.describe())['count']
pclass_survived = df.groupby(['pclass', 'survived']).survived.describe()
pclass_survived = (pclass_survived[pclass_survived['top'] == 1])['freq']
pclass_survived = pclass_survived.reset_index('survived').drop('survived', axis=1)
stats = pd.concat([class_tot, pclass_survived], axis=1)
prop = stats['freq'] / stats['count']

l = df.set_index('pclass').index.categories
plt.pie(list(prop), labels=list(l))
plt.show()
